# Лабораторная работа №2

## Задание 1. 
###### Провести классификацию найденного датасета, методом машины опорных векторов. В формате Markdown писать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

датасет: [Homicide Reports, 1980-2014](https://www.kaggle.com/datasets/murderaccountability/homicide-reports)

### Загрузка Библиотек

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

### Загрузка данных и предварительная обработка
###### Загрузка данных

In [2]:
N      = 20000
N_test = 0.996

In [3]:
data = pd.read_csv('database.csv',low_memory=False)
subset_data = data.sample(n=N, random_state=42) # выбираем 11000 строк
# Исключение ненужных столбцов
columns_to_drop = ['Record ID', 'Agency Code', 'Agency Name', 'Agency Type', 'City', 'State', 'Year', 'Month', 'Record Source']
subset_data = subset_data.drop(columns=columns_to_drop)

subset_data.dropna(inplace=True) # Удаление записей с пропущенными значениями
subset_data.head()

,Incident,Crime Type,Crime Solved,Victim Sex,Victim Age,Victim Race,Victim Ethnicity,Perpetrator Sex,Perpetrator Age,Perpetrator Race,Perpetrator Ethnicity,Relationship,Weapon,Victim Count,Perpetrator Count
484984,1,Murder or Manslaughter,No,Male,18,White,Hispanic,Unknown,0,Unknown,Unknown,Unknown,Handgun,1,0
608445,3,Murder or Manslaughter,No,Male,27,White,Not Hispanic,Unknown,0,Unknown,Unknown,Unknown,Firearm,0,0
417612,1,Murder or Manslaughter,Yes,Male,42,Black,Not Hispanic,Male,18,Black,Not Hispanic,Acquaintance,Handgun,0,0
372692,6,Murder or Manslaughter,No,Male,44,White,Not Hispanic,Unknown,0,Unknown,Unknown,Unknown,Handgun,0,0
402028,1,Murder or Manslaughter,Yes,Female,15,White,Unknown,Male,20,White,Unknown,Girlfriend,Rifle,0,0


In [4]:
# Выбор признаков и целевой переменной
X = subset_data.drop(columns=['Crime Solved'])
y = subset_data['Crime Solved']

In [5]:
X = pd.get_dummies(X)

In [6]:
# Масштабирование данных
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

### Разделение данных на тренировочную и тестовую выборки

In [7]:
# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=N_test, random_state=42)

### Построение и оценка модели

In [8]:
# Определение параметров для GridSearchCV
param_grid = {
    'C': range(1,100), 
    'gamma': ['scale',100, 10, 1, 0.1, 0.01], 
    'kernel': ['rbf', 'linear', 'poly', 'sigmoid']
}


In [9]:
# Использование GridSearchCV для поиска лучших параметров
grid_svc = GridSearchCV(
    SVC(), 
    param_grid, 
    cv=5, 
    scoring='accuracy',
    n_jobs=16
)

In [10]:
%%time
# Обучение модели
grid_svc.fit(X_train, y_train)

CPU times: total: 15.6 s
Wall time: 43 s


GridSearchCV(cv=5, estimator=SVC(), n_jobs=16,
             param_grid={'C': range(1, 100),
                         'gamma': ['scale', 100, 10, 1, 0.1, 0.01],
                         'kernel': ['rbf', 'linear', 'poly', 'sigmoid']},
             scoring='accuracy')

In [11]:
# Наилучшие параметры
print(f'Лучшие параметры: {grid_svc.best_params_}')
print(f'Лучшая точность: {grid_svc.best_score_}')

Лучшие параметры: {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}
Лучшая точность: 1.0


In [12]:
# Оценка качества модели на тестовых данных
print(grid_svc.score(X_test, y_test))

0.9701305220883534


In [13]:
%%time
# Получение предсказаний на тестовых данных
best_svc = grid_svc.best_estimator_
y_pred = best_svc.predict(X_test)

CPU times: total: 344 ms
Wall time: 367 ms


In [14]:
# Вывод classification report
print(classification_report(y_test, y_pred))
print("Матрица ошибок:\n", confusion_matrix(y_test, y_pred))
print("Точность:", accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

          No       0.99      0.91      0.95      5948
         Yes       0.96      1.00      0.98     13972

    accuracy                           0.97     19920
   macro avg       0.98      0.95      0.96     19920
weighted avg       0.97      0.97      0.97     19920

Матрица ошибок:
 [[ 5413   535]
 [   60 13912]]
Точность: 0.9701305220883534
